In [0]:
from google.colab import files

uploaded = files.upload()  

In [0]:
from pathlib import Path
from pytesseract import image_to_string
from PIL import Image
import os

imgs = Path('img')
imgs.mkdir()
os.system('pdftocairo libro.pdf img/img -png')


strings = []
for imgName in sorted(imgs.glob('*.png')):
    img =  Image.open(imgName)
    string = image_to_string( img, lang='spa' )
    strings.append(string)
    print(imgName,len(string))
    
from docx import Document
document = Document()
document.add_paragraph('\n'.join(strings))
document.save('libro.docx')

with open('libro.txt','w') as f:
    f.write('\n'.join(strings))

# Tesseract

In [0]:
!sudo apt install tesseract-ocr
!sudo apt install tesseract-ocr-spa
!sudo apt install libtesseract-dev

!pip install pytesseract
from PIL import Image
from pytesseract import image_to_string

In [0]:
im = Image.open( 'carpeta_maru-1-1.png' )
# im = Image.open( 'example.jpeg' )
im
# print( image_to_string(im) )
# print( image_to_string(im, lang='eng') )

In [0]:
string = image_to_string( im, lang='spa' )
print( string )

In [0]:
im2 = Image.open( 'example.png' )
im2

In [0]:
string2 = image_to_string( im2, lang='spa' )
print(string2)

# OpenCV2

In [0]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


In [0]:
img = cv2.imread('example.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
Image.fromarray( gray )

In [0]:
# Now we split the image to 5000 cells, each 20x20 size
cells = [np.hsplit(row,100) for row in np.vsplit(gray,50)]

# Make it into a Numpy array. It size will be (50,100,20,20)
x = np.array(cells)

# Now we prepare train_data and test_data.
train = x[:,:50].reshape(-1,400).astype(np.float32) # Size = (2500,400)
test = x[:,50:100].reshape(-1,400).astype(np.float32) # Size = (2500,400)

# Create labels for train and test data
k = np.arange(10)
train_labels = np.repeat(k,250)[:,np.newaxis]
test_labels = train_labels.copy()

# Initiate kNN, train the data, then test it with test data for k=1
knn = cv2.KNearest()
knn.train(train,train_labels)
ret,result,neighbours,dist = knn.find_nearest(test,k=5)

# Now we check the accuracy of classification
# For that, compare the result with test_labels and check which are wrong
matches = result==test_labels
correct = np.count_nonzero(matches)
accuracy = correct*100.0/result.size
print accuracy